In [5]:
import numpy as np
import pandas as pd
import torch
from torchvision import transforms, models, datasets
from waggle.plugin import Plugin
from waggle.data.vision import Camera

In [6]:
device =torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
root = ""
transformation = transforms.Compose(transforms.Resize((224,224)),
                                    transforms.ToTensor())
data = datasets.ImageFolder(root = root, transform=transformation)
test_size = int(0.2 * len(data))
train_size = len(data) - test_size
train_set, test_set = torch.utils.data.random_split(data, [train_size, test_size])


In [9]:
#Create additional layers for snow classification
model = models.resnet50(pretrained = True)
for param in model.parameters():
    param.requires_grad = False   
print(model.fc)
model.fc = torch.nn.Sequential(
    torch.nn.Linear(2048, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256,1)
)
model.to(device)

c:\Users\alxto\Anaconda3\envs\sage\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\alxto\Anaconda3\envs\sage\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Linear(in_features=2048, out_features=1000, bias=True)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
#parameters
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr = 0.003)
lr = 0.0001
batch_size = 128
epochs = 500
train_loader = torch.utils.data.DataLoader(train_set, batch_size= batch_size, shuffle= True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_set, batch_size= batch_size, shuffle= False, num_workers=8)

In [ ]:
#training loop
loss = 0 
train_loss, test_loss = [],[]

for epoch in range(epochs):
    